In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras import Sequential
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [ ]:
df = pd.read_csv('../input/heart-disease-uci/heart.csv')

In [ ]:
df.columns.isnull()

In [ ]:
plt.matshow(df.corr())
plt.show()

In [ ]:
x_df = df.iloc[:,:-1]

In [ ]:
y_df = df.iloc[:,-1]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(x_df,y_df,test_size=0.1,random_state=42)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(128,activation='selu',input_shape=[13]))
    model.add(Dropout(0.4))
    model.add(Dense(64,activation='selu'))
    model.add(Dropout(0.4))
    model.add(Dense(32,activation='selu'))
    model.add(Dropout(0.4))
    model.add(Dense(16,activation='selu'))
    model.add(Dropout(0.4))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

## Implement GridSearchCV on Keras

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Wrap Keras model so it can be used by scikit-learn
nn = KerasClassifier(build_fn=create_model, verbose=0)

# Create Hyperparameter space
epochs = [10,50,100]
batches = [5,10,100]
optimizers = ['rmsprop','adam']

# Create Hyperparameter options 
hyperparameters = dict(optimizer=optimizers, epochs=epochs,batch_size=batches)

# Create GridSearch
grid = GridSearchCV(estimator=nn, param_grid=hyperparameters)

grid_result = grid.fit(X_train,y_train)

In [ ]:
print(grid_result.best_params_)

In [ ]:
optimizer_best = grid_result.best_params_['optimizer']
epochs_best = grid_result.best_params_['epochs']

In [ ]:
model = create_model(optimizer_best)
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=epochs_best,verbose=0)

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epoch = range(len(acc))

plt.figure(figsize=(16,8))

plt.subplot(1,2,1)
plt.plot(epoch,acc,label='acc')
plt.plot(epoch,val_acc,label='val_acc')
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(epoch,loss,label='loss')
plt.plot(epoch,val_loss,label='val_loss')
plt.title('Loss')


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = (y_pred > 0.5)
print(confusion_matrix(y_pred, y_test))

### Compare predicted result with testset.

In [ ]:
print(np.concatenate((y_pred.reshape(len(y_pred),1), np.array(y_test).reshape(len(y_test),1)),1))